Author: Bram van Duinen

Date last edited: 17 nov 2023

Content: This notebook takes the co_occurences calculated in calc_co-occur and plots these on maps for each countries. Figures are saved in the Figures folder.

In [53]:
import matplotlib.pyplot as plt
import matplotlib
import cartopy
from cartopy.io import shapereader
import cartopy.crs as ccrs
import geopandas
import numpy as np
from tqdm.notebook import tqdm

In [48]:
co_occurrences = np.load('../Results/co_occurrences.npy')

In [54]:
basecountry_id = 0
# get natural earth data (http://www.naturalearthdata.com/)
for basecountry_id in tqdm(range(len(co_occurrences))):
    # get country borders
    resolution = '10m'
    category = 'cultural'
    name = 'admin_0_countries'
    shpfilename = shapereader.natural_earth(resolution, category, name)

    # read the shapefile using geopandas
    df = geopandas.read_file(shpfilename)

    # Set up the canvas
    fig = plt.figure(figsize=(8, 8))
    central_lon, central_lat = 0, 45
    extent = [-10, 45, 35, 70]
    ax = plt.axes(projection=cartopy.crs.Orthographic(central_lon, central_lat))
    ax.set_extent(extent)
    ax.gridlines()

    # Add natural earth features and borders
    ax.add_feature(cartopy.feature.BORDERS, linestyle=':', alpha=1)
    ax.add_feature(cartopy.feature.OCEAN, facecolor=("lightblue"))
    ax.add_feature(cartopy.feature.LAND)
    ax.coastlines(resolution='10m')

    # Insert your lists of countries and lag times here
    countries = ['AUT', 'BGR', 'BIH', 'CHE', 'CZE', 'ESP', 'FIN', 'FRA', 'GBR',
        'GRC', 'HRV', 'IRL', 'ITA', 'LVA', 'MKD', 'MNE', 'NOR', 'POL',
        'PRT', 'ROU', 'SRB', 'SVK', 'SVN', 'SWE']

    co_occur = co_occurrences[basecountry_id]

    # Normalise the lag times to between 0 and 1 to extract the colour
    co_occur_norm = (co_occur-np.nanmin(co_occur))/(np.nanmax(co_occur) - np.nanmin(co_occur))

    # Choose your colourmap here
    cmap = matplotlib.cm.get_cmap('viridis')

    for country, lag_norm in zip(countries, co_occur_norm):
        # read the borders of the country in this loop
        poly = df.loc[df['ISO_A3_EH'] == country]['geometry'].values[0]
        # get the color for this country
        rgba = cmap(lag_norm)
        # plot the country on a map
        ax.add_geometries(poly, crs=ccrs.PlateCarree(), facecolor=rgba, edgecolor='none', zorder=1)

    poly = df.loc[df['ISO_A3_EH'] == countries[basecountry_id]]['geometry'].values[0]
    ax.add_geometries(poly, crs=ccrs.PlateCarree(), facecolor='k', edgecolor='none', zorder=1)
    # Add a scatter plot of the original data so the colorbar has the correct numbers. Hacky but it works
    dummy_scat = ax.scatter(co_occur, co_occur, c=co_occur, cmap=cmap, zorder=0)
    fig.colorbar(mappable=dummy_scat, label='Co-occurences of 1-in-10 yr events', orientation='horizontal', shrink=0.8)
    plt.savefig(f'../Results/Figures/{countries[basecountry_id]}_Co-occurrences.png', dpi=300, bbox_inches='tight')
    plt.close()

  0%|          | 0/24 [00:00<?, ?it/s]

/tmp/ipykernel_4079849/621761602.py:38: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = matplotlib.cm.get_cmap('viridis')
/tmp/ipykernel_4079849/621761602.py:38: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = matplotlib.cm.get_cmap('viridis')
/tmp/ipykernel_4079849/621761602.py:38: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = matplotlib.cm.get_cmap('viridis')
/tmp/ipykernel_4079849/621761602.py:38: MatplotlibDeprecationWarning: The get_cmap function was depr